In [1]:
import logging
import os

In [2]:
%pwd

'/Users/sid/Desktop/MyProjects/End-to-end-ML-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/sid/Desktop/MyProjects/End-to-end-ML-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.mlProject.logger import logging
from src.mlProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} download! with following info: \n{headers}")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2024-07-04 15:55:41,273 ] 75 root - INFO - yaml file: config/config.yaml loaded successfully
[ 2024-07-04 15:55:41,275 ] 75 root - INFO - yaml file: params.yaml loaded successfully
[ 2024-07-04 15:55:41,279 ] 75 root - INFO - yaml file: schema.yaml loaded successfully
[ 2024-07-04 15:55:41,280 ] 95 root - INFO - created directory at: artifacts
[ 2024-07-04 15:55:41,281 ] 95 root - INFO - created directory at: artifacts/data_ingestion
[ 2024-07-04 15:55:41,282 ] 15 root - INFO - File already exists of size: ~ 23 KB
